In [2]:
import pandas as pd
import numpy as np
from ortools.sat.python import cp_model
from datetime import datetime, timedelta

In [3]:
import sys
sys.path.append('/home/cayena/Projeto_thefamily/TeacherScheduler/')

In [4]:
from streamlit_app.utils import transform_classes_dateframe,transform_teacher_dataframe,transform_alocation_dataframe,base_selection,expand_rows,replicate_row,clean_data
from streamlit_app.teacher_alocation import TeacherScheduler
from streamlit_app.validador import validador

2025-04-02 18:26:25.843 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-02 18:26:25.857 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-02 18:26:25.864 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-02 18:26:25.874 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [5]:
aulas_raw = pd.read_excel('/home/cayena/Downloads/ROTA ABRIL .xlsx')

professores_raw = pd.read_excel('/home/cayena/Downloads/TEACHERS ABRIL essa.xlsx')

In [6]:
df_result =transform_classes_dateframe(aulas_raw)
df_professor = transform_teacher_dataframe(professores_raw)

In [6]:
Ts = TeacherScheduler(df_result, df_professor)
base_alocada = Ts.schedule_teachers()
base_alocada

451


,professores_alocados,nome grupo
0,WANDERSON - ESP,GABRIELA VIP CONV EAD - 2X
1,DUDU,DALLAS EAD - 3X
2,MATTHEW,CELEBRATION EAD - 3X
3,FERNANDO,MIAMI EAD - 4X
4,BRUNO MORGILLO,TEXAS EAD - 3X
...,...,...
138,FELIPE,NEBRASKA P - 3X
139,CARLOS,NORTHAMPTOM P - 3X
140,BELLA,PHILADELPHIA P - 3X
141,FERNANDO,SOUTHAMPTON P - 3X


In [7]:
df_final,aulas_nao_alocadas = transform_alocation_dataframe(aulas_raw,base_alocada)
aulas_nao_alocadas

,nome grupo,horario,unidade,dias da semana,stage,livro,modalidade,grupo,n aulas,parag atual grupo,...,rescisao,permuta,bolsista,n_total_alunos,stage_formula,parag_final_formula,num_1_sempre,ultimo_professor,penultimo_professor,restricoes_professor


In [9]:
numero_aulas = df_final.groupby('teacher').agg({'n aulas':'sum'}).sort_values('n aulas',ascending=False).reset_index()
numero_aulas = pd.merge(numero_aulas,df_professor[['TEACHER','MEDIA']],left_on='teacher',right_on='TEACHER',how='left')
numero_aulas

,teacher,n aulas,TEACHER,MEDIA
0,BRUNO MORGILLO,27,BRUNO MORGILLO,24
1,LEONEL,22,LEONEL,24
2,BELLA,22,BELLA,24
3,MARIA,22,MARIA,24
4,FERNANDO,22,FERNANDO,24
5,RAPHAEL,21,RAPHAEL,24
6,CLAUDIO,18,CLAUDIO,20
7,DARLENE,18,DARLENE,16
8,FELIPE,17,FELIPE,20
9,MADDIE,17,MADDIE,20


In [7]:
base_gerada = pd.read_excel('/home/cayena/Downloads/rotas_geradas(11).xlsx')

In [11]:
numero_aulas = base_gerada.groupby('teacher').agg({'n aulas':'sum'}).sort_values('n aulas',ascending=False).reset_index()
numero_aulas = pd.merge(numero_aulas,df_professor[['TEACHER','MEDIA']],left_on='teacher',right_on='TEACHER',how='left')
numero_aulas.rename(columns={'n aulas':'aulas_alocadas'},inplace=True)
numero_aulas['diff_aulas'] = numero_aulas['aulas_alocadas'] - numero_aulas['MEDIA']
numero_aulas = numero_aulas[['teacher','MEDIA','aulas_alocadas','diff_aulas']]
numero_aulas

,teacher,MEDIA,aulas_alocadas,diff_aulas
0,BRUNO MORGILLO,24,27,3
1,RAPHAEL,24,25,1
2,BELLA,24,22,-2
3,MARIA,24,22,-2
4,LEONEL,24,21,-3
5,FERNANDO,24,21,-3
6,BARBARA,16,18,2
7,CLAUDIO,20,18,-2
8,FELIPE,20,17,-3
9,MADDIE,20,17,-3
